In [ ]:
"""
Найдите топ-40 самых длинных фильмов, аренда которых составляет больше 2 долларов. 
Выведите на экран название фильма (поле title), цену аренды (поле rental_rate), длительность фильма 
(поле length) и возрастной рейтинг (поле rating).
"""
SELECT title, 
       rental_rate, 
       length,
       rating
FROM movie
WHERE rental_rate > 2
ORDER BY length DESC
LIMIT 40;

In [ ]:
"""
Проанализируйте данные о возрастных рейтингах отобранных фильмов. Выгрузите в итоговую таблицу следующие поля:
возрастной рейтинг (поле rating);
минимальное и максимальное значения длительности (поле length); назовите поля min_length и max_length соответственно;
среднее значение длительности (поле length); назовите поле avg_length;
минимум, максимум и среднее для цены просмотра (поле rental_rate); назовите поля min_rental_rate, max_rental_rate, avg_rental_rate соответственно.
Отсортируйте среднюю длительность фильма по возрастанию.
"""
SELECT rating,
       MIN(length) AS min_length,
       MAX(length) AS max_length,
       AVG(length) AS avg_length,
       MIN(rental_rate) AS min_rental_rate,
       MAX(rental_rate) AS max_rental_rate,
       AVG(rental_rate) AS avg_rental_rate 
FROM (SELECT title, 
             rental_rate, 
             length,
             rating
      FROM movie
      WHERE rental_rate > 2
      ORDER BY length DESC
      LIMIT 40) AS top
GROUP BY rating
ORDER BY avg_length;

In [ ]:
"""
Найдите средние значения полей, в которых указаны минимальная и максимальная длительность отобранных фильмов. 
Отобразите только два этих поля. Назовите их avg_min_length и avg_max_length соответственно.
"""
SELECT AVG(n.min_length) AS avg_min_length,
       AVG(n.max_length) AS avg_max_length
FROM (SELECT top.rating,
            MIN(top.length) AS min_length,
            MAX(top.length) AS max_length,
            AVG(top.length) AS avg_length,
            MIN(top.rental_rate) AS min_rental_rate,
            MAX(top.rental_rate) AS max_rental_rate,
            AVG(top.rental_rate) AS avg_rental_rate
     FROM
       (SELECT title,
               rental_rate,
               length,
               rating
        FROM movie
        WHERE rental_rate > 2
        ORDER BY length DESC
       LIMIT 40) AS top
    GROUP BY top.rating
    ORDER BY avg_length) AS n;

In [ ]:
"""
Отберите альбомы, названия которых содержат слово 'Rock' и его производные. 
В этих альбомах должно быть восемь или более треков. 
Выведите на экран одно число — среднее количество композиций в отобранных альбомах.
"""
SELECT AVG(alb.count)
FROM (SELECT album.album_id,
             COUNT(track_id) AS count
      FROM album 
      LEFT JOIN track 
          ON track.album_id = album.album_id
      WHERE title LIKE '%Rock%'
      GROUP BY album.album_id
      HAVING COUNT(track_id) > 7) AS alb;

In [ ]:
"""
Для каждой страны посчитайте среднюю стоимость заказов в 2009 году по месяцам. 
Отберите данные за 2, 5, 7 и 10 месяцы и сложите средние значения стоимости заказов. 
Выведите названия стран, у которых это число превышает 10 долларов.
"""
SELECT billing_country
FROM (SELECT billing_country,
             EXTRACT(MONTH FROM CAST(invoice_date AS date)) AS invoice_month,
             AVG(total) AS avg_t
     FROM invoice
     WHERE EXTRACT(MONTH FROM CAST(invoice_date AS date)) IN (2, 5, 7, 10)
       AND EXTRACT(YEAR FROM CAST(invoice_date AS date)) = 2009
     GROUP BY billing_country,
              invoice_month) AS new
GROUP BY billing_country
HAVING SUM(new.avg_t) > 10;

In [ ]:
"""
Для каждой страны (поле billing_country) посчитайте минимальное, максимальное и среднее значение выручки из поля total. 
Назовите поля так: min_total, max_total и avg_total. Нужные поля для выгрузки хранит таблица invoice. 
При подсчёте учитывайте только те заказы, которые включают более пяти треков. 
Стоимость заказа должна превышать среднюю цену одного трека. Используйте код, написанный в предыдущих заданиях. 
Отсортируйте итоговую таблицу по значению в поле avg_total от большего к меньшему.
"""
SELECT billing_country,
       MIN(total) AS min_total,
       MAX(total) AS max_total,
       AVG(total) AS avg_total
FROM invoice
WHERE invoice_id IN (SELECT invoice_id
                    FROM invoice_line 
                    GROUP BY invoice_id
                    HAVING COUNT(track_id) > 5)
GROUP BY billing_country
HAVING AVG(total) > (SELECT AVG(unit_price)
                    FROM invoice_line)
ORDER BY avg_total DESC;

In [ ]:
"""
Отберите десять самых коротких по продолжительности треков и выгрузите названия их жанров.
"""
SELECT name
FROM genre
WHERE genre_id IN (
                    SELECT genre_id
                    FROM track
                    GROUP BY track_id
                    ORDER BY milliseconds
                    LIMIT 10);

In [ ]:
"""
Выгрузите уникальные названия городов, в которых стоимость заказов превышает среднее значение за 2009 год.
"""
SELECT DISTINCT billing_city
FROM invoice
GROUP BY billing_city
HAVING SUM(total) > (SELECT AVG(total)
                    FROM invoice
                    WHERE EXTRACT(YEAR FROM CAST(invoice_date AS date)) = 2009);

In [ ]:
"""
Найдите возрастной рейтинг с самыми дорогими для аренды фильмами. 
Для этого посчитайте среднюю стоимость аренды фильма каждого рейтинга. 
Выведите на экран названия категорий фильмов с этим рейтингом. 
Добавьте второе поле со средним значением продолжительности фильмов.
"""
SELECT DISTINCT category.name,
                AVG(length)
FROM movie
LEFT JOIN film_category
    ON film_category.film_id = movie.film_id
LEFT JOIN category
    ON category.category_id = film_category.category_id
WHERE movie.rating = (SELECT rating
                      FROM movie
                      GROUP BY rating
                      ORDER BY AVG(rental_rate) DESC
                      LIMIT 1)
GROUP BY category.name;

In [ ]:
"""
Составьте сводную таблицу. Посчитайте заказы, оформленные за каждый месяц в течение нескольких лет: с 2011 по 2013 год. 
Итоговая таблица должна включать четыре поля: invoice_month, year_2011, year_2012, year_2013. Поле invoice_month должно 
хранить месяц в виде числа от 1 до 12. 
Если в какой-либо месяц заказы не оформляли, номер такого месяца всё равно должен попасть в таблицу.
"""
SELECT y_2011.invoice_month, 
       year_2011,
       year_2012,
       year_2013
FROM (SELECT EXTRACT(MONTH FROM CAST(invoice_date AS date)) AS invoice_month,
             COUNT(total) AS year_2011
      FROM invoice
      WHERE EXTRACT(YEAR FROM CAST(invoice_date AS date)) = 2011
      GROUP BY invoice_month) AS y_2011
LEFT JOIN 
    (SELECT EXTRACT(MONTH FROM CAST(invoice_date AS date)) AS invoice_month,
            COUNT(total) AS year_2012
     FROM invoice
     WHERE EXTRACT(YEAR FROM CAST(invoice_date AS date)) = 2012
     GROUP BY invoice_month) AS y_2012 ON y_2011.invoice_month = y_2012.invoice_month
LEFT JOIN 
    (SELECT EXTRACT(MONTH FROM CAST(invoice_date AS date)) AS invoice_month,
            COUNT(total) AS year_2013
     FROM invoice
     WHERE EXTRACT(YEAR FROM CAST(invoice_date AS date)) = 2013
     GROUP BY invoice_month) AS y_2013 ON y_2011.invoice_month = y_2013.invoice_month;

In [ ]:
"""
Отберите фамилии пользователей, которые:
оформили хотя бы один заказ в январе 2013 года,
а также сделали хотя бы один заказ в остальные месяцы того же года.
Пользователей, которые оформили заказы только в январе, а в остальное время ничего не заказывали, в таблицу включать не нужно.
"""
SELECT client.last_name
FROM 
    (SELECT customer_id
     FROM invoice
     WHERE EXTRACT(YEAR FROM CAST(invoice_date AS date)) = 2013
       AND EXTRACT(MONTH FROM CAST(invoice_date AS date)) = 1
     GROUP BY customer_id) AS b
INNER JOIN 
    (SELECT customer_id
     FROM invoice
     WHERE EXTRACT(YEAR FROM CAST(invoice_date AS date)) = 2013
       AND EXTRACT(MONTH FROM CAST(invoice_date AS date)) <> 1
     GROUP BY customer_id) AS f ON b.customer_id = f.customer_id
LEFT JOIN client
    ON client.customer_id = b.customer_id;

In [ ]:
"""
Сформируйте статистику по категориям фильмов. Отобразите в итоговой таблице два поля:
название категории;
число фильмов из этой категории.
Фильмы для второго поля нужно отобрать по условию. 
Посчитайте фильмы только с теми актёрами и актрисами, которые больше семи раз снимались в фильмах, вышедших после 2013 года. 
Назовите поля name_category и total_films соответственно. 
Отсортируйте таблицу по количеству фильмов от большего к меньшему, 
а затем по полю с названием категории в лексикографическом порядке.
"""
SELECT DISTINCT name AS name_category,
       COUNT(*) AS total_films 
FROM film_category
INNER JOIN
    (SELECT DISTINCT movie.film_id
     FROM movie
     INNER JOIN film_actor
         ON film_actor.film_id = movie.film_id
     WHERE actor_id IN 
         (SELECT actor.actor_id
          FROM actor
          INNER JOIN film_actor
              ON actor.actor_id = film_actor.actor_id
          INNER JOIN movie
              ON film_actor.film_id = movie.film_id
          WHERE movie.release_year > 2013
          GROUP BY actor.actor_id
          HAVING COUNT(film_actor.film_id) > 7)) AS n ON n.film_id = film_category.film_id
LEFT JOIN category
    ON category.category_id = film_category.category_id
GROUP BY name
ORDER BY total_films DESC, name_category;

In [ ]:
"""
Определите, летом какого года общая выручка в магазине была максимальной. Затем проанализируйте данные за этот год по странам. 
Выгрузите таблицу с полями:
country — название страны;
total_invoice — число заказов, оформленных в этой стране в тот год, когда общая выручка за лето была максимальной;
total_customer — число клиентов, зарегистрированных в этой стране.
Отсортируйте таблицу по убыванию значений в поле total_invoice, а затем добавьте сортировку по названию страны в 
лексикографическом порядке.
"""
SELECT DISTINCT billing_country AS country,
       COUNT(total) AS total_invoice,
       AVG(cl.count_t) AS total_customer
FROM invoice
LEFT JOIN (SELECT COUNT(customer_id) AS count_t,
                  country
           FROM client
           GROUP BY country) AS cl ON cl.country = invoice.billing_country
WHERE EXTRACT(YEAR FROM CAST(invoice_date AS date)) = 
                                    (SELECT EXTRACT(YEAR FROM CAST(invoice_date AS date)) AS idate
                                    FROM invoice
                                    WHERE EXTRACT(MONTH FROM CAST(invoice_date AS date)) IN (6, 7, 8)
                                    GROUP BY idate
                                    ORDER BY SUM(total) DESC
                                    LIMIT 1)
GROUP BY billing_country
ORDER BY total_invoice DESC, country;

In [ ]:
"""
Выведите топ-40 самых длинных фильмов, аренда которых составляет больше 2 долларов. 
Проанализируйте данные о возрастных рейтингах отобранных фильмов. Выгрузите в итоговую таблицу следующие поля:
возрастной рейтинг (поле rating);
минимальное и максимальное значения длительности (поле length), назовите поля min_length и max_length соответственно;
среднее значение длительности (поле length), назовите поле avg_length;
минимум, максимум и среднее для цены просмотра (поле rental_rate), 
назовите поля min_rental_rate, max_rental_rate, avg_rental_rate соответственно.
Отсортируйте среднюю длительность фильма по возрастанию.
"""
WITH
top AS (SELECT *
        FROM movie
        WHERE rental_rate > 2
        ORDER BY length DESC
        LIMIT 40)

SELECT rating,
       MIN(length) AS min_length,
       MAX(length) AS max_length,
       AVG(length) AS avg_length,
       MIN(rental_rate) AS min_rental_rate,
       MAX(rental_rate) AS max_rental_rate,
       AVG(rental_rate) AS avg_rental_rate 
FROM top
GROUP BY rating
ORDER BY avg_length;

In [ ]:
"""
Проанализируйте данные из таблицы invoice за 2012 и 2013 годы. В итоговую таблицу должны войти поля:
month — номер месяца;
sum_total_2012 — выручка за этот месяц в 2012 году;
sum_total_2013 — выручка за этот месяц в 2013 году;
perc — процент, который отображает, насколько изменилась месячная выручка в 2013 году по сравнению с 2012 годом.
Округлите значение в поле perc до ближайшего целого числа. Отсортируйте таблицу по значению в поле month от меньшего к большему.
"""
WITH
i_2012 AS (SELECT EXTRACT(MONTH FROM CAST(invoice_date AS date)) AS month_2012,
                  SUM(total) AS stotal
           FROM invoice
           WHERE EXTRACT(YEAR FROM CAST(invoice_date AS date)) = 2012
           GROUP BY month_2012),
i_2013 AS (SELECT EXTRACT(MONTH FROM CAST(invoice_date AS date)) AS month_2013,
                  SUM(total) AS stotal
           FROM invoice
           WHERE EXTRACT(YEAR FROM CAST(invoice_date AS date)) = 2013
           GROUP BY month_2013)

SELECT month_2012 AS month,
       i_2012.stotal AS sum_total_2012,
       i_2013.stotal AS sum_total_2013,
       ROUND(i_2013.stotal / i_2012.stotal * 100 - 100) AS perc
FROM i_2012
LEFT JOIN i_2013
    ON i_2012.month_2012 = i_2013.month_2013
ORDER BY month;